# Dask in Python 3 (Ubuntu Linux)

https://docs.dask.org/en/latest/

In [1]:
import dask
dask.__version__

'2.21.0'

In [2]:
import distributed
distributed.__version__

'2.21.0'

In [3]:
import dask
import dask.distributed
import os
dask.config.set({
    'temporary_directory': os.path.expanduser('~/tmp'),
    'scheduler.work-stealing': True
})

In [4]:
dask.config.config

{'temporary-directory': '/home/user/tmp',
 'dataframe': {'shuffle-compression': None},
 'array': {'svg': {'size': 120}},
 'optimization': {'fuse': {'active': True,
   'ave-width': 1,
   'max-width': None,
   'max-height': inf,
   'max-depth-new-edges': None,
   'subgraphs': None,
   'rename-keys': True}},
 'distributed': {'version': 2,
  'scheduler': {'allowed-failures': 3,
   'bandwidth': 100000000,
   'blocked-handlers': [],
   'default-data-size': '1kiB',
   'events-cleanup-delay': '1h',
   'idle-timeout': None,
   'transition-log-length': 100000,
   'work-stealing': True,
   'work-stealing-interval': '100ms',
   'worker-ttl': None,
   'pickle': True,
   'preload': [],
   'preload-argv': [],
   'unknown-task-duration': '500ms',
   'default-task-durations': {'rechunk-split': '1us', 'shuffle-split': '1us'},
   'validate': False,
   'dashboard': {'status': {'task-stream-length': 1000},
    'tasks': {'task-stream-length': 100000},
    'tls': {'ca-file': None, 'key': None, 'cert': None},

In [6]:
from dask.distributed import Client
client = Client('127.0.0.1:8786')
client

Client Scheduler: tcp://127.0.0.1:8786 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 256.00 MB


Start `dash-scheduler` and `dash-worker` with `--dashboard-prefix b9bacd7b-6cee-402c-88ed-9d74b07f29a1/port/8787`


The dashboard is actually at `https://cocalc.com/{{ THE PROJECT UUID }}/port/8787/status`

Websocket forwarding doesn't work, though ... hmm...

alternatively, start an X11 desktop in cocalc and run `google-chrome` at http://127.0.0.1:8787/status

### data array similar to numpy arrays

In [7]:
import dask.array as da

In [8]:
client.restart()

x = da.random.random((1000, 1000), chunks=(300, 300))
x

dask.array<random_sample, shape=(1000, 1000), dtype=float64, chunksize=(300, 300), chunktype=numpy.ndarray>

In [9]:
x = x.rechunk({0: 500, 1: 500}).persist()
x

dask.array<rechunk-merge, shape=(1000, 1000), dtype=float64, chunksize=(500, 500), chunktype=numpy.ndarray>

In [10]:
client.scatter(x)

<Future: finished, type: dask.Array, key: Array-a43378b13ba8382373df14b6dd7f61d4>

In [11]:
y = x + x.T
z = y[::50, 100:].mean(axis=1)
z

dask.array<mean_agg-aggregate, shape=(20,), dtype=float64, chunksize=(10,), chunktype=numpy.ndarray>

In [12]:
z.shape

(20,)

In [12]:
out = z.compute()
out

array([0.99706421, 1.03277113, 0.9993643 , 0.99231638, 1.02139168,
       0.98631926, 0.99280159, 0.97743904, 0.99200793, 1.00457694,
       1.01779522, 0.98179355, 1.01977627, 1.01330775, 1.00401255,
       0.98929948, 0.98495306, 0.99648525, 0.98166991, 1.01806776])

In [13]:
rz = (z[:100].sum() / z[:100].shape[0])
rz.compute()

0.9974348104666977

In [14]:
del x, y, z, rz, out

### sum 1000 ints

In [12]:
client.restart()

import dask.bag as db
nums = db.from_sequence(range(1234), npartitions=10)
nums.sum().compute()

760761

### functions and native lists

In [17]:
import numpy as np
xx = dask.array.array(np.arange(100))
xx = xx.reshape(10, -1)
xx = xx.rechunk(2, 2)
xx = client.persist(xx)
xx

dask.array<rechunk-merge, shape=(10, 10), dtype=int64, chunksize=(2, 2), chunktype=numpy.ndarray>

In [18]:
y = client.submit(lambda x : x.dot(x.T), xx)
y.result()

dask.array<sum-aggregate, shape=(10, 10), dtype=int64, chunksize=(2, 2), chunktype=numpy.ndarray>

In [18]:
client.restart()

import dask.array as da
from time import sleep
from random import random
import numpy as np

delay = lambda : sleep(.2 * random())

def formula(x):
    delay()
    return x**3 - x**2 + 1


def neg(a):
    delay()
    return -a

def dup(a):
    delay()
    return 2 * a

def mysum(a, b):
    delay()
    return lambda a, b: a + b

N = 100
A = client.map(formula, range(N))
B = client.map(neg, A[:N // 3])
B.extend(client.map(dup, A[N // 3:2 * N // 3]))
B.extend(B[2*N//3:])
C = client.map(mysum, A, B)
total = client.submit(sum, B)
total.result()

8212721

### loops?

In [19]:
from time import sleep
from random import random

@dask.delayed
def fib(x):
    sleep(random())
    if x <= 1:
        return 1
    else:
        a, b = fib(x - 2), fib(x - 1)
        a, b = dask.compute(a, b)
        return a + b

In [20]:
dask.compute((fib(i) for i in range(5)), 10)

([1, 1, 2, 3, 5], 10)

## Dask Bags

In [21]:
import dask.bag as db
b1 = db.from_sequence(range(-1031, 1000), npartitions=50)
b1

dask.bag<from_sequence, npartitions=50>

In [22]:
import operator

In [23]:
is_odd = lambda x : x % 2 == 0

#b1.groupby(is_odd).map(lambda k_v : (k_v[0], sum(k_v[1]))).compute()
b1.foldby(is_odd, operator.add, 0).compute()

[(False, -16256), (True, -16240)]

## Dask Delayed

In [14]:
from dask import delayed

In [15]:
inc = lambda x : x+1
from operator import add

In [16]:
from time import sleep
z = delayed(0)
for i in range(5):
    x = delayed(inc)(i)
    y = delayed(inc)(delayed(add)(i, x))
    z = delayed(add)(z, y)
z.compute()

30

In [17]:
z.vizualize(filename=os.path.expanduser('~/dask-delayed-1.png'), format='png')

Delayed('vizualize-ad3fba49-bb55-4757-a0ce-668e7fb3aac8')

In [18]:
z.vizualize()

Delayed('vizualize-3dde23e6-bc93-4e01-a407-6b5d33a0ddbe')

## Dask ML

https://dask-ml.readthedocs.io/en/latest/

In [0]:
import dask_ml
dask_ml.__version__

In [0]:
from dask_ml.preprocessing import Categorizer, DummyEncoder
from dask_ml.linear_model import LogisticRegression

In [0]:
lr = LogisticRegression()
lr

In [0]:
from joblib import parallel_backend
with parallel_backend('dask') as pb:
    print(pb[0])

In [0]:
import joblib
joblib.__version__

In [34]:
client.restart()

Client Scheduler: tcp://127.0.0.1:8786 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 3 Cores: 3 Memory: 768.00 MB


## Ad-hoc Local Cluster

In [34]:
from dask.distributed import Client, LocalCluster
import dask.array as da

cluster = LocalCluster(
    n_workers=3,
    threads_per_worker=1,
    processes=True,
    diagnostics_port=None,
)
client = Client(cluster)
x = da.random.random((300, 300), chunks=(10, 10))
y = x + x.T
z = (y.mean(axis=1) / y.shape[0]).std()
print(z.compute())

/usr/local/lib/python3.6/dist-packages/distributed/dashboard/core.py:79: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


7.34683839787855e-05
